**Install Dependencies**

In [ ]:
!pip install -q flask pyngrok transformers accelerate sentencepiece einops bitsandbytes

**Hugging Face Login**

In [ ]:
from huggingface_hub import login

# Use your existing HF token (same as original code)
login("INPUT_YOUR_HuggingFaceHub_TOKEN_HERE")

**Create app.py**

In [ ]:
%%writefile app.py
# ==========================================
# 🧠 AI Travel Planner Chatbot — app.py
# ==========================================
# - Loads Llama 3.1 8B Instruct in 4-bit
# - Defines ask_bot() (same logic as your original)
# - Exposes a Flask web UI for travel planning
# ==========================================

from flask import Flask, render_template, request
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# --------------------------------------------------
# 1️⃣ Load LLM in 4-bit (Meta-Llama 3.1 8B Instruct)
# --------------------------------------------------
model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"

# Tokenizer (unchanged)
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    load_in_4bit=True   # 👈 4-bit loading enabled (instead of load_in_8bit)
)

print("🚀 Model Loaded in 4-bit mode successfully!")


# --------------------------------------------------
# 2️⃣ Core Generation Function — ask_bot (unchanged)
# --------------------------------------------------
def ask_bot(user_prompt, max_tokens=1500, temperature=0.7, top_p=0.9):
    """
    Call the LLM with your strict travel-planner system message
    and return only the generated portion.
    (Logic preserved exactly from your original code.)
    """
    messages = [
        {
            "role": "system",
            "content": (
                "You are a professional AI Travel Planner. Provide accurate, realistic, "
                "well-structured travel plans while strictly following the rules and output format. "
                "Never repeat or restate user input or instructions. Always start directly with the answer."
            )
        },
        {"role": "user", "content": user_prompt}
    ]

    # Build chat prompt correctly using the chat template
    prompt_text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    # Tokenize and move to model device
    inputs = tokenizer(prompt_text, return_tensors="pt").to(model.device)

    # Generate completion
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_tokens,
        temperature=temperature,
        top_p=top_p,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

    # Get ONLY newly generated tokens (exclude prompt tokens)
    generated_tokens = outputs[0][inputs["input_ids"].shape[1]:]

    # Decode only new tokens
    final_output = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()

    return final_output


# --------------------------------------------------
# 3️⃣ Flask App Setup
# --------------------------------------------------
app = Flask(__name__)


# --------------------------------------------------
# 4️⃣ Home Route — Show Input Form + Result
# --------------------------------------------------
@app.route("/", methods=["GET", "POST"])
def home():
    """
    Renders the main page.
    On POST: collects form inputs, builds your big dynamic prompt,
             calls ask_bot(), and returns the result.
    """
    output = ""

    if request.method == "POST":
        # ===== Collect User Inputs from Web Form =====
        destination = request.form.get("destination", "").strip()
        dates = request.form.get("dates", "").strip()
        budget_raw = request.form.get("budget", "").strip()
        group_type = request.form.get("group_type", "").strip()
        style = request.form.get("style", "").strip()

        # Safe conversion for budget (fallback to raw string if needed)
        try:
            budget_value = float(budget_raw)
        except ValueError:
            budget_value = budget_raw  # if user types non-numeric, we just pass as-is

        # ===== Build Your Original Dynamic Prompt (unchanged text) =====
        prompt = f"""
YOU ARE A STRICT TRAVEL ITINERARY GENERATOR. FOLLOW ALL RULES EXACTLY.
ALL VALUES IN {{currency_symbol}}.

========================
USER INPUTS (DYNAMIC)
========================
Destination: {destination}
Travel Dates: {dates}
Total Budget: ${budget_value}
Group Type: {group_type}
Travel Style: {style}

========================
GLOBAL BEHAVIOR RULES
========================
- Never explain, never describe rules, never add extra text.
- Only generate the structured output defined below.
- All sections, bullets, headings, and line formats MUST match exactly.
- All computations MUST use the dynamic values above.
- Absolutely no placeholders in the final output.

========================
BUDGET RULES (DYNAMIC + STRICT)
========================
1. Total Budget = {{budget}}
2. Safety Reserve %:
   - Use {{safety_reserve_low}}% if destination = low risk
   - Use {{safety_reserve_high}}% if destination = high risk
   - If unknown, use {{safety_reserve_default}}%
3. Safety Reserve = Total Budget * selected %
4. Usable Budget = Total Budget – Safety Reserve
5. Safety Reserve must NOT appear in planning sections.

========================
BUDGET ALLOCATION RULES (MANDATORY)
========================
All category percentages MUST stay within their ranges and MUST total exactly 100%, adjusting in order (Misc → Transport → Activities → Food → Accommodation) without breaking any limits.
* Accommodation: 25% to 40%
* Food & Drinks: 25% to 35%
* Activities & Entry Fees: 15% to 25%
* Transportation: 10% to 15%
* Miscellaneous: 0% to 10%

MANDATORY:
- Values MAY vary dynamically but MUST be within the min/max provided.
- Allocation sum MUST equal exactly 100% of the Usable Budget.
- No leftover. No overshoot.
- If rounding causes mismatch, adjust in this order:
  Misc → Transport → Activities → Food → Accommodation
  (never break min/max bounds)

========================
DYNAMIC DAY-COST RULES
========================
- Must create exactly {{num_days}} travel days (MANDATORY).
- Must not include any farewell days after the actual trip days.
- Estimated Day Cost distribution must use these logical dynamic weights:
  {{day_weights_list}}
- Adjust final day to fix rounding differences.
- Sum of all day costs MUST equal Usable Budget exactly.

========================
ACTIVITY RULES (DYNAMIC)
========================
For EACH day:
- Create a unique day theme (use destination + style + day activities)
- Morning / Afternoon / Evening MUST contain only **activity descriptions**
- These activities MUST NOT include any form of food/drink reference
- After the 3 periods, list Breakfast, Lunch, Dinner separately
- Meals MUST be destination-specific, style-aligned, and non-repetitive
  unless repetition is logically justified (e.g., famous dish)

Prohibited inside activities (case-insensitive):
food, drink, cafe, restaurant, meal, breakfast, lunch, dinner, snacks, bar, pub, tasting, dining, cuisine

If an activity normally involves food, rewrite it as a cultural, scenic, or experiential activity.

========================
FOOD SECTION RULES
========================
- Place Food & Restaurant Suggestions **immediately before** Budget Breakdown.
- Provide between {{food_suggestions_min}} and {{food_suggestions_max}} items.
- Items must be unique, destination-specific, and suitable for {{style}} style.

========================
BUDGET BREAKDOWN RULES
========================
- Display ONCE ONLY.
- Must follow EXACT format:

* Accommodation: {{currency_symbol}}{{amount}} ({{percent}}%)
* Food & Drinks: {{currency_symbol}}{{amount}} ({{percent}}%)
* Activities & Entry Fees: {{currency_symbol}}{{amount}} ({{percent}}%)
* Transportation: {{currency_symbol}}{{amount}} ({{percent}}%)
* Miscellaneous: {{currency_symbol}}{{amount}} ({{percent}}%)

Percent values must be precise and derived from actual allocations.

========================
TRAVEL TIPS (MANDATORY)
========================
- Provide EXACTLY 3 tips.
- Must be destination-specific and safety-focused.
- Each tip = one bullet only.
- After the 3rd tip: STOP. No closing sentence.

========================
FINAL OUTPUT FORMAT (FOLLOW EXACTLY)
========================

### Trip Details

#### Budget Calculation

* Total Budget: {{currency_symbol}}{{budget}}
* Safety Reserve: {{currency_symbol}}{{safety_reserve}} ({{safety_reserve_pct}}% of Total Budget)
* Usable Budget: {{currency_symbol}}{{usable_budget}}

========================
DAY FORMAT (APPLIED PER DAY)
========================
For each day from Day 1 to Day {{num_days}}, use EXACTLY this format:

**Day {{day_number}}: {{day_theme}} — Estimated Day Cost: {{currency_symbol}}{{day_cost}}**

* Morning: {{morning_activity}}
* Afternoon: {{afternoon_activity}}
* Evening: {{evening_activity}}
* Breakfast: {{breakfast}}
* Lunch: {{lunch}}
* Dinner: {{dinner}}


### Food & Restaurant Suggestions
{{food_suggestions}}

### Budget Breakdown
{{budget_breakdown}}

### Travel Tips
* {{tip1}}
* {{tip2}}
* {{tip3}}
"""

        # ===== Call LLM via ask_bot() =====
        output = ask_bot(prompt)

    # Render template with (possibly empty) output
    return render_template("index.html", output=output)


# --------------------------------------------------
# 5️⃣ Run Flask App (for local/debug; in Colab we use nohup)
# --------------------------------------------------
if __name__ == "__main__":
    app.run(host="0.0.0.0", port=8000, debug=False)

**Create Template & Static Folders**

In [ ]:
!mkdir -p templates
!mkdir -p static

**HTML - Template**

In [ ]:
%%writefile templates/index.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8" />
    <title>🌍 AI Travel Planner Chatbot</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}" />
</head>
<body>
    <div class="hero-section">
        <div class="overlay"></div>

        <div class="hero-content">
            <h1>🌍 AI Travel Planner</h1>
            <p>Generate smart, budget-aware itineraries tailored to your style.</p>

            <form method="POST" class="input-card">
                <input type="text" name="destination" placeholder="Destination (e.g., Bali)" required />
                <input type="text" name="dates" placeholder="Travel dates (e.g., 12 June – 15 June)" required />
                <input type="text" name="budget" placeholder="Total budget (e.g., 600)" required />

                <select name="group_type" required>
                    <option value="" disabled selected>Who are you traveling with?</option>
                    <option value="Solo">Solo</option>
                    <option value="Friends">Friends</option>
                    <option value="Family">Family</option>
                    <option value="Couple">Couple</option>
                </select>

                <select name="style" required>
                    <option value="" disabled selected>Preferred travel style</option>
                    <option value="Adventure">Adventure</option>
                    <option value="Food">Food</option>
                    <option value="Relaxing">Relaxing</option>
                    <option value="Culture">Culture</option>
                    <option value="Nightlife">Nightlife</option>
                    <option value="Mix">Mix</option>
                </select>

                <button type="submit" class="btn-generate">Generate Itinerary ✈️</button>
            </form>
        </div>
    </div>

    {% if output %}
    <div class="result-section fade-in">
        <div class="result-card">
            <h2>🧳 Your AI-Generated Travel Plan</h2>
            <p style="white-space: pre-line;">{{ output }}</p>
        </div>
    </div>
    {% endif %}
</body>
</html>

**CSS - Template**

In [ ]:
%%writefile static/style.css
/* ==========================================
   🌈 Basic styling for AI Travel Planner UI
   ========================================== */

@import url('https://fonts.googleapis.com/css2?family=Poppins:wght@400;600&display=swap');

body {
    margin: 0;
    font-family: 'Poppins', sans-serif;
    color: #fff;
}

/* Background hero section with travel theme */
.hero-section {
    position: relative;
    height: 100vh;
    display: flex;
    align-items: center;
    padding-left: 8%;
    background: url('https://images.pexels.com/photos/346885/pexels-photo-346885.jpeg')
        center/cover no-repeat;
}

.overlay {
    position: absolute;
    inset: 0;
    background: rgba(0,0,0,0.55);
}

/* Main glass card */
.hero-content {
    position: relative;
    z-index: 2;
    width: 520px;
    max-width: 90%;
    padding: 40px 45px;
    background: rgba(255,255,255,0.18);
    border-radius: 25px;
    backdrop-filter: blur(8px);
    box-shadow: 0 10px 35px rgba(0,0,0,0.45);
}

.hero-content h1 {
    font-size: 2.4rem;
    color: #ffe8a6;
    margin-bottom: 10px;
}

.hero-content p {
    font-size: 0.98rem;
    color: #e7ecff;
    margin-bottom: 20px;
}

/* Form layout */
.input-card {
    display: flex;
    flex-direction: column;
    gap: 12px;
}

input, select {
    width: 100%;
    padding: 11px 12px;
    border-radius: 10px;
    border: none;
    font-size: 0.95rem;
    background: rgba(255,255,255,0.92);
    color: #111;
}

/* Button */
.btn-generate {
    margin-top: 4px;
    padding: 12px;
    border: none;
    border-radius: 10px;
    font-weight: 600;
    font-size: 1rem;
    cursor: pointer;
    background: linear-gradient(135deg, #ff9800, #ff5722);
    color: #fff;
    box-shadow: 0 0 18px rgba(255,152,0,0.55);
    transition: 0.25s ease;
}

.btn-generate:hover {
    transform: translateY(-2px);
    box-shadow: 0 0 25px rgba(255,152,0,0.75);
}

/* Result section */
.result-section {
    background: #0d111f;
    padding: 60px 20px;
    display: flex;
    justify-content: center;
}

.result-card {
    max-width: 900px;
    background: rgba(255,255,255,0.08);
    padding: 25px 30px;
    border-radius: 16px;
    color: #e2e5ff;
    box-shadow: 0 8px 25px rgba(0,0,0,0.45);
}

/* Fade-in animation */
.fade-in {
    animation: fadeInUp 0.6s ease forwards;
}

@keyframes fadeInUp {
    from { opacity: 0; transform: translateY(20px); }
    to   { opacity: 1; transform: translateY(0); }
}

**Flask + NGROK**

In [ ]:
# ==========================================
# 🧹 Kill Previous Flask/ngrok (if any)
# ==========================================
!pkill -f flask || echo "No flask running"
!pkill -f ngrok || echo "No ngrok running"



In [ ]:
!lsof -i :8000

In [ ]:
# ==========================================
# 🚀 Start Flask (background) + Ngrok Tunnel
# ==========================================
!nohup python app.py > flask.log 2>&1 &


In [ ]:
from pyngrok import ngrok, conf

# Include your NGROK Auth token
conf.get_default().auth_token = "INPUT_YOUR_NGROK_TOKEN_HERE"

# Close previous tunnels if any
for t in ngrok.get_tunnels():
    try:
        ngrok.disconnect(t.public_url)
    except:
        pass

public_url = ngrok.connect(8000)
print("🌍 Public URL:", public_url)

# Show last few log lines to confirm app is running
!sleep 3 && tail -n 20 flask.log